# Group 1 Capstone Project Work
### Make sure to make / checkout your own branch before making changes

# Maddie Hince

In [7]:
import pandas as pd
import sqlite3

movie_gross_df = pd.read_csv('zippedData/bom.movie_gross.csv.gz') # Elijah
movie_info_df = pd.read_csv('zippedData/rt.movie_info.tsv.gz', delimiter='\t') # Cat
reviews_df = pd.read_csv('zippedData/rt.reviews.tsv.gz', delimiter='\t', encoding='latin') # Maddie
movies_df = pd.read_csv('zippedData/tmdb.movies.csv.gz') # Kilmar
movie_budgets_df = pd.read_csv('zippedData/tn.movie_budgets.csv.gz') # Michael

conn = sqlite3.connect('im.db') # Luis
# might want to read tables into dataframes later with pd.read_sql_table()

In [8]:
cursor = conn.cursor()
cursor.fetchall()

[]

# Cat Murad

# Elijah Soba

# Luis Estrada 

# Kilmar Lazo 

# Michael Eugene